In [1]:
from pathlib import Path
from src.utils import atoms2cfg, read_cfg
from ase.io import read

In [2]:
current = Path().cwd()
outcar_path = current.parents[1] / "mlip-3/test/examples/00.convert_vasp_outcar/OUTCAR"
traj = read(outcar_path, index=":")
file = "./test.cfg"
for i,atoms in enumerate(traj):
    append = False if i==0 else True
    atoms2cfg(atoms, file=file, unique_elements=["Ag"], append=append)
    

In [5]:
a = list(read_cfg(file="./test.cfg", symbols=["Ag"]))
b = read_cfg(file="./executable.cfg", symbols=["Ag"])

In [6]:
a

[Atoms(symbols='Ag32', pbc=True, cell=[[8.458526959, 1.656720733, 1.673168563], [1.573491987, 8.821430406, -1.494376014], [1.591543182, -1.495192313, 8.82509342]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.464367542, 1.654782691, 1.677485389], [1.570870501, 8.830852271, -1.499033421], [1.596387395, -1.500887298, 8.836299561]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.466931764, 1.653931823, 1.679380628], [1.569719577, 8.834988801, -1.501078187], [1.598514175, -1.503387596, 8.841219451]], calculator=SinglePointCalculator(...))]

In [6]:
b

Atoms(symbols='H32', pbc=True, cell=[[8.458527, 1.656721, 1.673169], [1.573492, 8.82143, -1.494376], [1.591543, -1.495192, 8.825093]], calculator=SinglePointCalculator(...))

In [7]:
a[2].get_stress()

array([ 1.63028165e-03,  1.63509737e-03,  1.27361234e-03,  3.03100848e-04,
       -2.93367316e-05, -2.87343416e-04])

In [8]:
traj[2].get_stress()

array([ 1.63027594e-03,  1.63510063e-03,  1.27360490e-03,  3.03100166e-04,
       -2.93288514e-05, -2.87346597e-04])

In [57]:
from ase import units
def get_virial_stress(atoms):
    """
    Calculate the virial stress tensor for an ASE Atoms object.

    Parameters:
    atoms (ase.Atoms): ASE Atoms object with calculator attached.

    Returns:
    stress (np.ndarray): Virial stress tensor in Voigt notation (6 components).
    """

    # Ensure the atoms object has a calculator

    # Get volume of the system
    volume = atoms.get_volume()

    # Get forces and positions
    forces = atoms.get_forces()
    positions = atoms.get_positions()

    # Initialize virial tensor
    virial = np.zeros((3, 3))

    # Calculate the virial contribution
    for i in range(len(atoms)):
        virial += np.outer(positions[i], forces[i])

    # Convert to stress tensor (Voigt notation)
    stress_voigt = np.zeros(6)
    stress_voigt[0] = virial[0, 0]
    stress_voigt[1] = virial[1, 1]
    stress_voigt[2] = virial[2, 2]
    stress_voigt[3] = virial[1, 2]
    stress_voigt[4] = virial[0, 2]
    stress_voigt[5] = virial[0, 1]

    # Normalize by volume and convert units from eV/Å³ to GPa
    # stress_voigt *= units.eV / (1e10 ** 3)
    stress_voigt /= volume

    return stress_voigt

In [58]:
get_virial_stress(atoms)

array([ 0.00026723,  0.00105734,  0.00107055, -0.00057019,  0.00031717,
       -0.0002914 ])